In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime, os
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
from tqdm import tqdm_notebook
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from utils import *

SEED = 42
seed_everything(SEED)

In [2]:
train = pd.read_pickle('../input/reduced/train_df.pkl')
test = pd.read_pickle('../input/reduced/test_df.pkl')

train_y = train['isFraud'].copy()

train_identity = pd.read_pickle('../input/reduced/train_identity.pkl')
test_identity = pd.read_pickle('../input/reduced/test_identity.pkl')

base_columns = list(train) + list(train_identity)
train_productCD = train['ProductCD']
test_productCD =test['ProductCD']

print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)
print('training identity size is ', train_identity.shape)
print('testing identity size is', test_identity.shape)

training data size is  (590540, 393)
training label size is (590540,)
testing data size is (506691, 393)
training identity size is  (144233, 42)
testing identity size is (141907, 42)


In [3]:
train = train.merge(train_identity, how='left', left_index=True, right_index=True)
test = test.merge(test_identity, how='left', left_index=True, right_index=True)

# train = reduce_mem_usage(train)
# test  = reduce_mem_usage(test)

print(train.shape)
print(test.shape)

(590540, 435)
(506691, 435)


In [4]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,id_33_0,id_33_1
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,9524.0,142.0,267648.0,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,122947.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0

# drop_columns

In [5]:
cols_to_drop = get_useless_columns(train)
# print(cols_to_drop)

More than 90% null: 12
More than 90% repeated value: 67


  ### above is from 0. data reduction

In [6]:
# deal with "card1" => train, test overlap elements
# remove noises
cat_cols = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']

for col in cat_cols: 
    valid_card = pd.concat([train[[col]], test[[col]]])
    valid_card = valid_card[col].value_counts()
    valid_card = valid_card[valid_card>2]
    valid_card = list(valid_card.index)

    train[col] = np.where(train[col].isin(test[col]), train[col], np.nan)
    test[col]  = np.where(test[col].isin(train[col]), test[col], np.nan)

    train[col] = np.where(train[col].isin(valid_card), train[col], np.nan)
    test[col]  = np.where(test[col].isin(valid_card), test[col], np.nan)

In [7]:
# TransactionAmt (doubt???)

# user ID
train['uid'] = train['card1'].astype(str)+'_'+train['card2'].astype(str)
test['uid'] = test['card1'].astype(str)+'_'+test['card2'].astype(str)

train['uid2'] = train['uid'].astype(str)+'_'+train['card3'].astype(str)+'_'+train['card4'].astype(str)
test['uid2'] = test['uid'].astype(str)+'_'+test['card3'].astype(str)+'_'+test['card4'].astype(str)

train['uid3'] = train['uid2'].astype(str)+'_'+train['addr1'].astype(str)+'_'+train['addr2'].astype(str)
test['uid3'] = test['uid2'].astype(str)+'_'+test['addr1'].astype(str)+'_'+test['addr2'].astype(str)

# Check if the Transaction Amount is common or not (we can use freq encoding here)
# In our dialog with a model we are telling to trust or not to these values   
train['TransactionAmt_check'] = np.where(train['TransactionAmt'].isin(test['TransactionAmt']), 1, 0)
test['TransactionAmt_check']  = np.where(test['TransactionAmt'].isin(train['TransactionAmt']), 1, 0)

# For our model current TransactionAmt is a noise
# https://www.kaggle.com/kyakovlev/ieee-check-noise
# (even if features importances are telling contrariwise)
# There are many unique values and model doesn't generalize well
# Lets do some aggregations
i_cols = ['card1','card2','card3','card5','uid','uid2','uid3']

for col in i_cols:
    for agg_type in ['mean','std']:
        new_col_name = col+'_TransactionAmt_'+agg_type
        temp_df = pd.concat([train[[col, 'TransactionAmt']], test[[col,'TransactionAmt']]])
        #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
        temp_df = temp_df.groupby([col])['TransactionAmt'].agg([agg_type]).reset_index().rename(
                                                columns={agg_type: new_col_name})
        
        temp_df.index = list(temp_df[col])
        temp_df = temp_df[new_col_name].to_dict()   
    
        train[new_col_name] = train[col].map(temp_df)
        test[new_col_name]  = test[col].map(temp_df)
           
# Small "hack" to transform distribution 
# (doesn't affect auc much, but I like it more)
# please see how distribution transformation can boost your score 
# (not our case but related)
# https://scikit-learn.org/stable/auto_examples/compose/plot_transformed_target.html
train['TransactionAmt'] = np.log1p(train['TransactionAmt'])
test['TransactionAmt'] = np.log1p(test['TransactionAmt'])        

train = train.replace(np.inf,999)
test = test.replace(np.inf,999)

In [8]:
# mark
# @tmp: delete later
# @target: target encoding later
# @freq: frequency encoding later

def fe(df):
    # deal with time variables
    df['DT'] = df['TransactionDT'].apply(
        lambda x: (START_DATE + datetime.timedelta(seconds=x)))  # tmp    
    df['DT_month'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
    df['DT_week_year'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
    df['DT_day_year'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear
    
    df['DT_hour'] = df['DT'].dt.hour
    df['DT_day_week'] = df['DT'].dt.dayofweek
    df['DT_day'] = df['DT'].dt.day
    # D9 column: same as hour
    df['D9'] = np.where(df['D9'].isna(), 0, 1)

    # transaction amount
    # done

    # ProductCD
    # separate models

    # card1: done above
    # card2-6

    # addr1-2

    # dist1-2

    # P_ & R_
    p = 'P_emaildomain'
    r = 'R_emaildomain'
    uknown = 'email_not_provided'
    
    df[p + '_bin'] = df[p].map(emails)
    df[r + '_bin'] = df[r].map(emails)
    df[p + '_suffix'] = df[p].map(lambda x: str(x).split('.')[-1])
    df[p + '_suffix'] = df[p + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    df[r + '_suffix'] = df[r].map(lambda x: str(x).split('.')[-1])
    df[r + '_suffix'] = df[r + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')    
    
    
    # P_ & R_

    df[p] = df[p].fillna(uknown)
    df[r] = df[r].fillna(uknown)
    df['email_check'] = np.where((df[p] == df[r]) & (df[p] != uknown), 1, 0)
    df[p + '_prefix'] = df[p].apply(lambda x: x.split('.')[0])
    df[r + '_prefix'] = df[r].apply(lambda x: x.split('.')[0])

    # C1-C14

    # D1 - D15

    # M1 - M9
    # except M4, others are binary
    m_cols = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']
    df['M_sum'] = df[m_cols].sum(axis=1).astype(np.int8)
    df['M_na'] = df[m_cols].isna().sum(axis=1).astype(np.int8)

    # V1 - V339

    # id1-11

    # id12-38

    #id30
    df['id_30'] = df['id_30'].fillna('unknown_device').str.lower()
    df['id_30_device'] = df['id_30'].apply(
        lambda x: ''.join([i for i in x if i.isalpha()]))
    df['id_30_version'] = df['id_30'].apply(
        lambda x: ''.join([i for i in x if i.isnumeric()]))

    # id31: to remove "id_31", use "lastest_browser"
    # id_31_device already freq encoded
    df = setBrowser(df)
    df['id_31'] = df['id_31'].fillna('unknown_device').str.lower()
    df['id_31_device'] = df['id_31'].apply(
        lambda x: ''.join([i for i in x if i.isalpha()]))
    device_map = train['id_31_device'].value_counts()
    device_map = device_map[device_map > 100]
    train['id_31_device'] = train['id_31_device'].map(device_map)

    # DeviceType, DeviceInfo  => to remove "DeviceInfo" > freq encoding
    df = setDevice(df)

    return df


train = fe(train)
test = fe(test)

In [9]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,id_33_0,id_33_1,uid,uid2,uid3,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,DT,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,P_emaildomain_bin,R_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_suffix,email_check,P_emaildomain_prefix,R_emaildomain_prefix,M_sum,M_na,id_30_device,id_30_version,lastest_browser,id_31_device,device_name
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,4.242188,W,13926.0,NaN,150.0,9524.0,142.0,267648.0,315.0,87.0,19.0,NaN,email_not_provided,email_not_provided,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,0,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,122947.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0

In [10]:
def nan_encode_func(train, test):
    df = pd.concat([train, test], axis=0)
    for col in tqdm_notebook(test.columns):
        nan_rate = df[col].isnull().sum() / len(df) 
        if((nan_rate> 0.2) & (nan_rate< 0.8)):
            train[col + "_nan_code"] = train[col].isnull().astype(int)
            test[col + "_nan_code"] = test[col].isnull().astype(int)
        
    return train, test
print("before shape is: ", train.shape)
train, test = nan_encode_func(train, test)
print("after shape is: ", train.shape)

before shape is:  (590540, 474)



after shape is:  (590540, 653)


In [11]:
# Kon: target mean encoding
# target_code_book = ['ProductCD', 'M4']
target_code_book = ['M4', 'DT_hour', 'DT_day_week', 'DT_day']
def target_encode_func(train, test, target_code_book):

    for col in tqdm_notebook(target_code_book):
        temp_dict = train.groupby([col])['isFraud'].agg([
            'mean'
        ]).reset_index().rename(columns={'mean': col + '_target_mean'})
        temp_dict.index = temp_dict[col].values
        temp_dict = temp_dict[col + '_target_mean'].to_dict()

        train[col + '_target_mean'] = train[col].map(temp_dict)
        test[col + '_target_mean'] = test[col].map(temp_dict)

    return train, test


train, test = target_encode_func(train, test, target_code_book)

In [12]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,id_33_0,id_33_1,uid,uid2,uid3,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,DT,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,P_emaildomain_bin,R_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_suffix,email_check,P_emaildomain_prefix,R_emaildomain_prefix,M_sum,M_na,id_30_device,id_30_version,lastest_browser,id_31_device,device_name,dist1_nan_code,D2_nan_code,D3_nan_code,D4_nan_code,D5_nan_code,D11_nan_code,M1_nan_code,M2_nan_code,M3_nan_code,M4_nan_code,M5_nan_code,M6_nan_code,M7_nan_code,M8_nan_code,M9_nan_code,V1_nan_code,V2_nan_code,V3_nan_code,V4_nan_code,V5_nan_code,V6_nan_code,V7_nan_code,V8_nan_code,V9_nan_code,V10_nan_code,V11_nan_code,V35_nan_code,V36_nan_code,V37_nan_code,V38_nan_code,V39_nan_code,V40_nan_code,V41_nan_code,V42_nan_code,V43_nan_code,V44_nan_code,V45_nan_code,V46_nan_code,V47_nan_code,V48_nan_code,V49_nan_code,V50_nan_code,V51_nan_code,V52_nan_code,V167_nan_code,V168_nan_code,V169_nan_code,V170_nan_code,V171_nan_code,V172_nan_code,V173_nan_code,V174_nan_code,V175_nan_code,V176_nan_code,V177_nan_code,V178_nan_code,V179_nan_code,V180_nan_code,V181_nan_code,V182_nan_code,V183_nan_code,V184_nan_code,V185_nan_code,V186_nan_code,V187_nan_code,V188_nan_code,V189_nan_code,V190_nan_code,V191_nan_code,V192_nan_code,V193_nan_code,V194_nan_code,V195_nan_code,V196_nan_code,V197_nan_code,V198_nan_code,V199_nan_code,V200_nan_code,V201_nan_code,V202_nan_code,V203_nan_code,V204_nan_code,V205_nan_code,V206_nan_code,V207_nan_code,V208_nan_code,V209_nan_code,V210_nan_code,V211_nan_code,V212_nan_code,V213_nan_code,V214_nan_code,V21

In [13]:
# Kon: freq encoding
freq_code_book = [
    'card1', 'card2', 'card3', 'card5', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6',
    'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'addr1', 'addr2',
    'dist1', 'dist2', 'P_emaildomain_bin', 'R_emaildomain_bin',
    'P_emaildomain_suffix', 'R_emaildomain_suffix', 'P_emaildomain_prefix',
    'R_emaildomain_prefix',
    'id_30_device', 'id_30_version', 'id_31_device', 'device_name'
]


def freq_encode_func(train, test, freq_code_book):

    for col in tqdm_notebook(freq_code_book):
        temp_df = pd.concat([train[[col]], test[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()
        train[col + '_fq_enc'] = train[col].map(fq_encode)
        test[col + '_fq_enc'] = test[col].map(fq_encode)

    return train, test


train, test = freq_encode_func(train, test, freq_code_book)

In [14]:
for col in ['DT_month','DT_week_year','DT_day_year']:
    temp_df = pd.concat([train[[col]], test[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
            
    train[col+'_total'] = train[col].map(fq_encode)
    test[col+'_total']  = test[col].map(fq_encode)
        

periods = ['DT_month','DT_week_year','DT_day_year']
i_cols = ['uid']
for period in periods:
    for col in i_cols:
        new_column = col + '_' + period
            
        temp_df = pd.concat([train[[col,period]], test[[col,period]]])
        temp_df[new_column] = temp_df[col].astype(str) + '_' + (temp_df[period]).astype(str)
        fq_encode = temp_df[new_column].value_counts().to_dict()
            
        train[new_column] = (train[col].astype(str) + '_' + train[period].astype(str)).map(fq_encode)
        test[new_column]  = (test[col].astype(str) + '_' + test[period].astype(str)).map(fq_encode)
        
        train[new_column] /= train[period+'_total']
        test[new_column]  /= test[period+'_total']

In [15]:
# Kon: Str encoding
str_rm_col = []
def str_encode_func(train, test):

    for col in tqdm_notebook(list(train)):
        if train[col].dtype == 'O':
            str_rm_col.append(col)
#             print(col)
#             train[col] = train[col].fillna('unseen_before_label')
#             test[col] = test[col].fillna('unseen_before_label')

#             train[col] = train[col].astype(str)
#             test[col] = test[col].astype(str)

#             le = LabelEncoder()
#             le.fit(list(train[col]) + list(test[col]))
#             train[col] = le.transform(train[col])
#             test[col] = le.transform(test[col])

#             train[col] = train[col].astype('category')
#             test[col] = test[col].astype('category')
    return train, test


train, test = str_encode_func(train, test)

In [16]:
train_DT_month = train['DT_month']
test_DT_month =test['DT_month']

In [17]:
manual_drop_cols = [
    'TransactionDT',
    'isFraud',
    'P_emaildomain',
    'R_emaildomain',
    'TransactionAmt',
    'uid',
    'uid2',
    'uid3',
    'DT',
    'DT_week_year',
    'DT_day_year',
    'DT_hour',
    'DT_day_week',
    'DT_day',
    'DeviceInfo',
    'DT_day_year_total',
    'DT_week_year_total',
    'DT_month_total',
    'id_30',
    'id_31',
    'id_33',
    'id_31_device'
]

In [18]:
drop_list = manual_drop_cols + cols_to_drop + str_rm_col + manual_drop_cols
drop_list = list(dict.fromkeys(drop_list))
print(drop_list)

['TransactionDT', 'isFraud', 'P_emaildomain', 'R_emaildomain', 'TransactionAmt', 'uid', 'uid2', 'uid3', 'DT', 'DT_week_year', 'DT_day_year', 'DT_hour', 'DT_day_week', 'DT_day', 'DeviceInfo', 'DT_day_year_total', 'DT_week_year_total', 'DT_month_total', 'id_30', 'id_31', 'id_33', 'id_31_device', 'id_23', 'V111', 'V301', 'V112', 'V132', 'C3', 'V293', 'V321', 'V286', 'V134', 'V296', 'V118', 'V284', 'V113', 'V104', 'id_24', 'V102', 'V114', 'dist2', 'V135', 'V316', 'V309', 'V124', 'V105', 'V298', 'V299', 'V281', 'id_08', 'V295', 'id_07', 'V116', 'V121', 'V108', 'V110', 'V300', 'V98', 'id_27', 'V120', 'V133', 'V137', 'V318', 'V101', 'V311', 'id_18', 'D7', 'id_21', 'V107', 'V109', 'V119', 'V297', 'V136', 'V320', 'V103', 'id_26', 'V115', 'V106', 'id_25', 'V117', 'V122', 'V290', 'V319', 'id_22', 'V129', 'V305', 'V123', 'V125', 'ProductCD', 'DeviceType', 'P_emaildomain_bin', 'R_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_suffix', 'P_emaildomain_prefix', 'R_emaildomain_prefix', 'id_30

In [19]:
print('before ---')
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

train = train.drop(drop_list,axis=1)
test = test.drop(drop_list,axis=1)

train = train.reset_index()
test = test.reset_index()
# train = train.drop(drop_list,axis=1)
# test = test.drop(drop_list,axis=1)

print('after ---')
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

before ---
training data size is  (590540, 695)
training label size is (590540,)
testing data size is (506691, 695)
after ---
training data size is  (590540, 597)
training label size is (590540,)
testing data size is (506691, 597)


In [25]:
########################### Features elimination 
from scipy.stats import ks_2samp
features_check = []
columns_to_check = set(list(train))#.difference(base_columns+rm_cols)
for i in columns_to_check:
    features_check.append(ks_2samp(test[i].fillna(-1), train[i].fillna(-1))[1])

features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
features_discard = list(features_check[features_check==0].index)
features_discard.remove('TransactionID')
features_discard.remove('DT_month')
print(features_discard)


['M9', 'V18', 'V2_nan_code', 'V6', 'V93', 'V44_nan_code', 'V39_nan_code', 'V42_nan_code', 'V38', 'V73', 'V37', 'V22', 'V28', 'V74', 'V85', 'V34', 'V59', 'V67', 'V70', 'V3', 'V26', 'V53', 'V49', 'V41', 'V62', 'V71', 'V11_nan_code', 'M9_nan_code', 'D4', 'V35', 'V42', 'id_31_device_fq_enc', 'D6', 'V45_nan_code', 'M_na', 'V75', 'V84', 'V43', 'V36', 'V14', 'D3', 'id_13_nan_code', 'V91', 'V90', 'M_sum', 'V10_nan_code', 'V35_nan_code', 'C12_fq_enc', 'V1', 'V29', 'V4_nan_code', 'V13', 'M2', 'M7', 'V43_nan_code', 'V1_nan_code', 'V88', 'V37_nan_code', 'V32', 'uid_DT_day_year', 'V4', 'V76', 'V94', 'id_34', 'V80', 'V27', 'V2', 'V81', 'V79', 'V50_nan_code', 'V69', 'V57', 'V3_nan_code', 'V89', 'V5', 'D14', 'V51', 'V25', 'V7', 'V77', 'id_13', 'V68', 'V16', 'M3', 'D10', 'V19', 'V33', 'V44', 'C12', 'D3_nan_code', 'V82', 'V8_nan_code', 'V7_nan_code', 'V52_nan_code', 'M2_nan_code', 'V52', 'V87', 'V31', 'V66', 'V58', 'V9', 'V17', 'V55', 'V36_nan_code', 'D5_nan_code', 'V78', 'V51_nan_code', 'D11_nan_code',

In [26]:
print('before ---')
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

train = train.drop(features_discard,axis=1)
test = test.drop(features_discard,axis=1)

print('after ---')
print('training data size is ', train.shape)
print('training label size is', train_y.shape)
print('testing data size is', test.shape)

before ---
training data size is  (590540, 597)
training label size is (590540,)
testing data size is (506691, 597)
after ---
training data size is  (590540, 439)
training label size is (590540,)
testing data size is (506691, 439)


In [27]:
train['ProductCD'] =train_productCD.values
test['ProductCD'] =test_productCD.values

train.head()

,TransactionID,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C13,C14,D1,D2,D8,D9,D12,M4,M5,M6,V95,V96,V97,V99,V100,V126,V127,V128,V130,V131,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V282,V283,V285,V287,V288,V289,V291,V292,V294,V302,V303,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_32,id_35,id_36,id_37,id_38,id_33_0,id_33_1,TransactionAmt_check,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid_TransactionAmt_mean,uid_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,DT_month,email_check,lastest_browser,dist1_nan_code,D2_nan_code,M4_nan_code,M5_nan_code,M6_nan_code,V167_nan_code,V168_nan_code,V169_nan_code,V170_nan_code,V171_nan_code,V172_nan_code,V173_nan_code,V174_nan_code,V175_nan_code,V176_nan_code,V177_nan_code,V178_nan_code,V179_nan_code,V180_nan_code,V181_nan_code,V182_nan_code,V183_nan_code,V184_nan_code,V185_nan_code,V186_nan_code,V187_nan_code,V188_nan_code,V189_nan_code,V190_nan_code,V191_nan_code,V192_nan_code,V193_nan_code,V194_nan_code,V195_nan_code,V196_nan_code,V197_nan_code,V198_nan_code,V199_nan_code,V200_nan_code,V201_nan_code,V202_nan_code,V203_nan_code,V204_nan_code,V205_nan_code,V206_nan_code,V207_nan_code,V208_nan_code,V209_nan_code,V210_nan_code,V211_nan_code,V212_nan_code,V213_nan_code,V214_nan_code,V215_nan_code,V216_nan_code,V217_nan_code,V218_nan_code,V219_nan_code,V220_nan_code,V221_nan_code,V222_nan_code,V223_nan_code,V224_nan_code,V225_nan_code,V226_nan_code,V227_nan_code,V228_nan_code,V229_nan_code,V230_nan_code,V231_nan_code,V232_nan_code,V233_nan_code,V234_nan_code,V235_nan_code,V236_nan_code,V237_nan_code,V238_nan_code,V239_nan_code,V240_nan_code,V241_nan_code,V242_nan_code,V243_nan_code,V244_nan_code,V245_nan_code,V246_nan_code,V247_nan_code,V248_nan_code,V249_nan_code,V250_nan_code,V251_nan_code,V252_nan_code,V253_nan_code,V254_nan_code,V255_nan_code,V256_nan_code,V257_nan_code,V258_nan_code,V259_nan_code,V260_nan_code,V261_nan_code,V262_nan_code,V263_nan_code,V264_nan_code,V265_nan_code,V266_nan_code,V267_nan_code,V268_nan_code,V269_nan_code,V270_nan_code,V271_nan_code,V272_nan_code,V273_nan_code,V274_nan_code,V275_nan_code,V276_nan_code,V277_nan_code,V278_nan_code,id_01_nan_code,id_02_nan_code,id_05_nan_code,id_06_nan_code,id_11_nan_code,id_12_nan_code,id_15_nan_code,id_16_nan_code,id_17_nan_code,id_19_nan_code,id_20_nan_code,id_28_nan_code,id_29_nan_code,id_33_nan_code,id_35_nan_code,id_36_nan_code,id_37_nan_code,id_38_nan_code,DeviceType_nan_code,id_33_0_nan_code,id_33_1_nan_code,R_emaildomain_bin_nan_code,M4_target_mean,DT_hour_target_mean,DT_day_week_target_mean,DT_day_target_mean,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,C1_fq_enc,C2_fq_enc,C3_fq_enc,C4_fq_enc,C5_fq_enc,C6_fq_enc,C7_fq_enc,C8_fq_enc,C9_fq_enc,C10_fq_enc,C11_fq_enc,C13_fq_enc,C14_fq_enc,addr1_fq_enc,addr2_fq_enc,dist1_fq_enc,dist2_fq_enc,P_emaildomain_bin_fq_enc,R_emaildomain_bin_fq_enc,P_email

In [28]:
train = reduce_mem_usage(train)
test  = reduce_mem_usage(test)
train.to_pickle('../input/reduced/train_df_features.pkl')
train_y.to_pickle('../input/reduced/train_y.pkl')
test.to_pickle('../input/reduced/test_df_features.pkl')

Mem. usage decreased to 493.91 Mb (59.3% reduction)
Mem. usage decreased to 431.51 Mb (58.8% reduction)
